# Content-Based Recommender

### What's in this Notebook? 

#### Section 1. Code for Recommender and Fuzzy Wuzzy Search
I use a sample of 20,000 cases from the original dataset since trying the entire dataset kills the kernel. Then, I use several functions for the recommender system (described below). The system identifies 10 recommended wines based on the Description. 

#### Section 2. Examining various Recommendations
I use Count Vectorizer on a few different recommendations to examine why those specific wines are being recommended. The system appears to be working in that the recommended wines share common words from the description. 

#### Section 3. Evaluation Metric
I created an evaluation metric to determine how successful the recommender was at identifying similar wines. I took the top 10 words (from the CVEC) found in the recommended wines and calculated the average number of times the words appeared. If the average number of times a top-10 word appeared was more than 7, this would suggest that at least seven of the wines shared those words in their descriptions. Across a sample of 100 recommendations, the average score was 6.7. 

#### Section 4. Recommenders for Red and White wines only
It did bug me a bit that someone could enter in a white or a champagne and the recommender might bring back several reds. So, I created separate dataframes for red and whites and duplicated the code. I envision the online version of this might start by asking users, "do you want a recommendation for a red or white, or either?" and that way the user could choose their path. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from wordcloud import WordCloud, STOPWORDS
from IPython.display import Image
#from IPython.display import Image
from os import path
from sklearn.cluster import KMeans

from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.metrics.pairwise import linear_kernel

plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, cross_val_score

In [3]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_colwidth', 999)

In [4]:
df = pd.read_csv('./wine.csv', index_col=0)

In [5]:
df.shape

(73691, 14)

### Experimenting with FuzzyWuzzy

Fuzzy wuzzy is a way to identify strings that do not match exactly. For example, if you ask people to write down their occupation, people might write "Data Scientist" or "data scientist" or "data science." All three are the same but a regular matching would see them as three different terms because they are not written exactly the same. 

Fuzzy wuzzy is useful for the wine recommender because a user will need to enter in the exact name of a wine, and often wine titles are very complex. For example: "Hirsch 2013 East Ridge Pinot Noir (Sonoma Coast)." Someone may not remember that Sonoma Coast is in parentheses, or that the 2013 is before East Ridge. 

Fuzzy wuzzy takes in a few parameters: Key (what the user puts in to search for), Choices (the list that will be searched through), the scoring system, and sometimes a Limit if you want to list several possible matches. 

In [6]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [7]:
key = "decoy cabernet"

choices = df['title']

#This code brings back several matches
matches = (process.extract(key, choices, scorer = fuzz.ratio, limit = 5))
print("\n".join(map(str, matches)))

#This code brings back just the top match
#print(process.extractOne(key, choices, scorer = fuzz.ratio, score_cutoff = 50)[0])

('Colcanyon 2010 Cabernet Sauvignon', 51, 14770)
('Decoy 2010 Cabernet Sauvignon (Napa County)', 50, 4455)
('Decoy 2009 Cabernet Sauvignon (Napa Valley)', 50, 68591)
('Decoy 2008 Cabernet Sauvignon (Napa Valley)', 50, 109078)
('Decoy 2016 Rosé (California)', 49, 40790)


## Section 1. Sampling and the Content-Based Recommender

The recommender uses Term Frequency Inverse Document Frequency (TF-IDF). TF-IDF down-weights more common words (e.g., fruit) so that the model gets the "bigger picture" of other words that may contribute to a recommendation. After fitting the descriptions to TF-IDF, pairwise similarity scores are calculated using cosine distance. Those similarities scores are stored in a matrix. An indices series is created that contains the wine titles and their indices. 

Next, a user is able to enter a wine title (an approximation), and three functions are used to identify recommendtions: 

1) Fuzzy wuzzy search to give five options for the approximate wine title entered. The user then picks which wine title he/she wants to use. I added this part because someone may not remember a wine title exactly, so this will help identify similar titles. 

2) Another fuzzy wuzzy search to identify the specific wine title. The exact wine title is needed since the recommender looks for the specific match in the dataframe.  

3) Get_recommendations. Takes the specific wine title, finds its index, finds wines that are most similar based on the similarity scores, sorts them, finds the top 10 (could change this), finds the indices of those 10 and matches those indices back to the title from the original sample dataframe, and brings back the 10 wines. 

Many thanks to this article from DataCamp for guidance on code for content-based recommenders: https://www.datacamp.com/community/tutorials/recommender-systems-python#comments

In [8]:
sample = df.sample(n=20000, random_state=42)

In [9]:
sample = sample.reset_index()

In [10]:
sample = sample.drop(['index'], axis = 1)

In [11]:
sample.shape

(20000, 14)

In [12]:
my_additional_stop_words = ['wine', 'flavor', 'flavors', 'notes', 'offers', 'enjoy', 'drink', 'bottling',
                           'really', 'reveal', 'picked', 'partner', 'thanks', 'grown', 'gain', 
                           'thanks', 'percentage', 'buy', 'mid', 'gain', 'bottle', 'amid', 'available',
                           'come', 'aspect', 'try', 'pinot', 'noir', 'chardonnay','cabernet', 'sauvignon',
                            'red','white','bordeaux', 'riesling', 'blanc','syrah','merlot',
                            'rose', 'zinfandel','malbec','nebbiolo','sangiovese', 'tempranillo','sparkling',
                            'rhone','gris', 'riesling', 'sirah', 'petite', 'petit', 'make']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

#I toyed with not including the varietal names as stop words, but it didn't appear to make much difference
# my_additional_stop_words = ['wine', 'flavor', 'flavors', 'notes', 'offers', 'enjoy', 'drink', 'bottling',
#                            'really', 'reveal', 'picked', 'partner', 'thanks', 'grown', 'gain', 
#                            'thanks', 'percentage', 'buy', 'mid', 'gain', 'bottle', 'amid', 'available',
#                            'come', 'aspect', 'try', 'make']
# stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

In [13]:
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]

In [14]:
tf = TfidfVectorizer(stop_words=stop_words,tokenizer=tokenize, max_features = 2000)
tf_matrix = tf.fit_transform(sample['description'])

In [21]:
tf_matrix

<20000x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 396966 stored elements in Compressed Sparse Row format>

In [15]:
cosine_sim = linear_kernel(tf_matrix, tf_matrix)

In [25]:
cosine = pd.DataFrame(cosine_sim)
cosine.head(15)

0         1         2         3         4         5         6      \
0   1.000000  0.011451  0.015741  0.007518  0.000000  0.000000  0.008807   
1   0.011451  1.000000  0.016025  0.007654  0.000000  0.015346  0.008967   
2   0.015741  0.016025  1.000000  0.010521  0.000000  0.058103  0.028852   
3   0.007518  0.007654  0.010521  1.000000  0.000000  0.049963  0.101328   
4   0.000000  0.000000  0.000000  0.000000  1.000000  0.102885  0.000000   
5   0.000000  0.015346  0.058103  0.049963  0.102885  1.000000  0.021585   
6   0.008807  0.008967  0.028852  0.101328  0.000000  0.021585  1.000000   
7   0.046729  0.109476  0.014043  0.036368  0.000000  0.094261  0.007858   
8   0.000000  0.044786  0.000000  0.044237  0.072384  0.010119  0.025146   
9   0.075163  0.043736  0.080684  0.036322  0.030286  0.011431  0.017560   
10  0.000000  0.000000  0.000000  0.019602  0.028256  0.057886  0.000000   
11  0.000000  0.013772  0.000000  0.043692  0.000000  0.000000  0.000000   
12  0.000000  0.000000  0.000000  0.081794  0.164225  0.029160  0.016548   
13  0.149241  0.031726  0.017440  0.008330  0.000000  0.000000  0.074096   
14  0.079061  0.012270  0.015018  0.034061  0.037982  0.054362  0.037992   

       7         8         9         10        11        12        13     \
0   0.046729  0.000000  0.075163  0.000000  0.000000  0.000000  0.149241   
1   0.109476  0.044786  0.043736  0.000000  0.013772  0.000000  0.031726   
2   0.014043  0.000000  0.080684  0.000000  0.000000  0.000000  0.017440   
3   0.036368  0.044237  0.036322  0.019602  0.043692  0.081794  0.008330   
4   0.000000  0.072384  0.030286  0.028256  0.000000  0.164225  0.000000   
5   0.094261  0.010119  0.011431  0.057886  0.000000  0.029160  0.000000   
6   0.007858  0.025146  0.017560  0.000000  0.000000  0.016548  0.074096   
7   1.000000  0.032458  0.008547  0.063445  0.071926  0.038330  0.011118   
8   0.032458  1.000000  0.037470  0.020708  0.119574  0.044436  0.059804   
9   0.008547  0.037470  1.000000  0.000000  0.060528  0.051991  0.026544   
10  0.063445  0.020708  0.000000  1.000000  0.082345  0.025331  0.000000   
11  0.071926  0.119574  0.060528  0.082345  1.000000  0.056462  0.086774   
12  0.038330  0.044436  0.051991  0.025331  0.056462  1.000000  0.000000   
13  0.011118  0.059804  0.026544  0.000000  0.086774  0.000000  1.000000   
14  0.036864  0.034746  0.050926  0.019649  0.000000  0.070457  0.000000   

       14        15        16        17        18        19        20     \
0   0.079061  0.099818  0.033415  0.065349  0.052203  0.050687  0.010162   
1   0.012270  0.067698  0.000000  0.038862  0.021374  0.000000  0.025926   
2   0.015018  0.029023  0.018830  0.018056  0.054954  0.000000  0.014222   
3   0.034061  0.038233  0.186290  0.028444  0.035600  0.028248  0.038528   
4   0.037982  0.042045  0.000000  0.000000  0.000000  0.000000  0.297870   
5   0.054362  0.059854  0.011974  0.045927  0.000000  0.054879  0.049183   
6   0.037992  0.043929  0.010536  0.026493  0.077867  0.000000  0.106939   
7   0.036864  0.090996  0.052847  0.059369  0.007490  0.047781  0.022720   
8   0.034746  0.034806  0.000000  0.054613  0.009013  0.000000  0.046480   
9   0.050926  0.054118  0.059890  0.087530  0.082591  0.026704  0.028876   
10  0.019649  0.000000  0.000000  0.090497  0.027965  0.118740  0.036288   
11  0.000000  0.081660  0.000000  0.034104  0.010098  0.000000  0.000000   
12  0.070457  0.025812  0.000000  0.054837  0.054528  0.000000  0.056126   
13  0.000000  0.072076  0.000000  0.038874  0.023262  0.036764  0.140013   
14  1.000000  0.088531  0.009574  0.009181  0.056388  0.050637  0.010890   

       21        22        23        24        25        26        27     \
0   0.052282  0.026856  0.014989  0.019053  0.036091  0.000000  0.000000   
1   0.017822  0.000000  0.120498  0.030320  0.000000  0.018355  0.000000   
2   0.008211  0.031685  0.020977  0.000000  0.000000  0.000000  0.000000   
3   0.070243  0.015253  0.010019  0.098000  0.007532

In [18]:
indices = pd.Series(sample.index, index=sample['title']).drop_duplicates()

### Function for the Recommender

In [16]:
pick_one = ''
def get_choices(key):
    
    choices = sample['title']    
    
    #gets the top five closest matches ands asks the user to pick one
    matches = (process.extract(key, choices, scorer = fuzz.ratio, limit = 5))
    print("\n Here are the 5 closest matches...Pick one for the recommender to use. \n")
    print("\n".join(map(str, matches)))
    
    global pick_one
    pick_one = input("Enter wine here: ")
    
    print("\n Okay, here are 10 other wines that are like", pick_one.title())
    
    def fuzzy_match(key):
    
        choices = sample['title']    

        #returns the top matching choice
        return process.extractOne(key, choices, scorer = fuzz.token_sort_ratio, score_cutoff = 50)[0]

    #assigning the top choice to title since that is an input for the next function
    title = fuzzy_match(pick_one)

    def get_recommendations2(title, cosine_sim=cosine_sim):

        # Get the index of the wine that matches the title
        idx = indices[title]

        # Get the pairwsie similarity scores of all wines with that wine
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort the wines based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the 10 most similar wines
        sim_scores = sim_scores[0:11] #included the 0-th index so I can see the original wine too

        # Get the wines indices
        wine_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar wines
        return sample['title'].iloc[wine_indices]

    results = get_recommendations2(title)
    print(results)

### This is where the user enters the wine for which they want recommendations.

In [ ]:
wine = input('What wine would you like to search for?')
get_choices(wine)

What wine would you like to search for?black box cabernet

 Here are the 5 closest matches...Pick one for the recommender to use. 

('Black Box 2011 Malbec (Mendoza)', 58, 8581)
('Black Box 2010 Cabernet Sauvignon (California)', 57, 18491)
('Black Box 2008 Merlot (California)', 55, 16969)
('Black Box 2009 Merlot (California)', 55, 18494)
('Black Cordon 2012 Cabernet Sauvignon (Napa Valley)', 48, 9141)


#### Original function
This is the original code for the recommender. I use this to examine some individual recommendations below, just because the code is shorter than the one I did above with the fuzzy wuzzy searching. 

In [33]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:11] #included the 0-th index so I can see the original wine too
    wine_indices = [i[0] for i in sim_scores]
    return sample['title'].iloc[wine_indices]

## Section 2. Examining a few recommendations to see how the system is working

#### Count Vectorizer on recommendation #1: Chardonnay
The input wine was a Chardonnay, and for the most part the recommender is bringing back whites, but also a few reds. This is not necessarily wrong, so I'll use Count Vectorizer to see common themes. The most common words used in this recommendation are: Crisp, Lively, Light, Acid, and Refreshing, along with references to Texture, and being good with Food. 

In [34]:
get_recommendations('Geyser Peak 2014 Chardonnay (California)')

1050                                                      Geyser Peak 2014 Chardonnay (California)
12588                                                        Peltier 2014 Hybrid Chardonnay (Lodi)
4882                                                                       Corte Giara 2016  Soave
3426                                              Hirsch 2013 East Ridge Pinot Noir (Sonoma Coast)
15477                                        Oak Farm 2015 Mohr-Fry Ranches Sauvignon Blanc (Lodi)
12879                                               Château des Antonins 2015 Rosé (Bordeaux Rosé)
13475                                                   Bogle 2014 Old Vine Zinfandel (California)
10320                                   Jean-Claude Debeaune 2013 Domaine Lenoir  (Mâcon-Villages)
4099                                                      Leese-Fitch 2013 Chardonnay (California)
8899     Château Philippe-le-Hardi 2011 Clos Philippe-le-Hardi  (Bourgogne Hautes Côtes de Beaune)
12566     

In [35]:
rec1 = sample.loc[[1050, 3426, 4099, 13475, 12879, 12588, 11, 4882, 15477, 10320, 8899],['description', 'title']]
rec1

,description,title
1050,"A crisp, lively texture makes this medium-bodied wine refreshing while light flavors of green apple and kiwi make it tasty. Vibrant acidity gives it freshness and will help it be versatile with food.",Geyser Peak 2014 Chardonnay (California)
3426,"Light cherry and strawberry aromas and flavors go with a medium-body and grippy texture to make this wine rather tight on the palate. It's a good, versatile lunch and dinner drink because the supportive tannins and acidity will pair nicely with food.",Hirsch 2013 East Ridge Pinot Noir (Sonoma Coast)
4099,"This shows fresh, crisp apple and lemon-cream aromas, with clean appley flavors and buttery accents. Medium bodied, it tends toward the soft, rather than crisp, side. Lively fruit flavors make it refreshing.",Leese-Fitch 2013 Chardonnay (California)
13475,Lively fruit flavors and great balance make this wine easy to appreciate and a versatile pairing with food. Fresh cherry and raspberry aromas meet rich flavors on a smooth texture that has just enough light tannins for a little grip.,Bogle 2014 Old Vine Zinfandel (California)
12879,"This fruity wine has a refreshing citrus flavor. Lightly textured, it is lively and crisp. A mineral edge gives it tight structure, putting the wine into the food category.",Château des Antonins 2015 Rosé (Bordeaux Rosé)
12588,"Light, appley aromas and lively apple and citrus flavors brighten this unoaky wine. It has refreshing acidity, an elegant texture and crisp finish.",Peltier 2014 Hybrid Chardonnay (Lodi)
11,"Crisp and appetizing lemon and grapefruit flavors do their job very well in this vivid, refreshing and medium-bodied wine. Its bright acidity makes the mouth water and will help it pair beautifully with a wide variety of foods.",Lake and Vine Cellars 2014 Sauvignon Blanc (Lake County)
4882,"Here's a lively, informal white redolent of green apple, peach and citrus, with vibrant acidity. It's simple but well made, with a crisp, refreshing finish.",Corte Giara 2016 Soave
15477,"Fresh and light in aroma, crisp and delicate in flavor, this wine is light-bodied, well-balanced and attractively restrained in character. Because of all that it will be extremely versatile with food—everything from shucked oysters to Thai food to smoked salmon.",Oak Farm 2015 Mohr-Fry Ranches Sauvignon Blanc (Lodi)
10320,"Tight, crisp and fruity, this is a refreshing and bright wine. Lively acidity cuts right through the lightly mineral texture. Full of fruit, fresh and lively, this should be drunk now.",Jean-Claude Debeaune 2013 Domaine Lenoir (Mâcon-Villages)


In [36]:
rec1_text = rec1['description']

In [50]:
cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize, max_features = 2000)

cvec.fit(rec1_text)

new_df  = pd.DataFrame(cvec.transform(rec1_text).todense(),
             columns=cvec.get_feature_names()).sum(axis=0)

new_df.transpose().sort_values(0, ascending=False).transpose().head(15)

crisp          11
live            9
light           8
refresh         7
textur          7
food            7
acid            7
aroma           6
fresh           6
appl            5
bodi            5
versatil        4
medium          4
medium bodi     4
fruit           3
dtype: int64

#### Recommendation 2: Another white recommendation - Spanish white
This time we see all white wines being recommended. Common words are Fruit, Clean, Finish, Simple, and Aroma. 

In [37]:
get_recommendations('Vega Sindoa 2015 Chardonnay (Navarra)')

19126                                                 Vega Sindoa 2015 Chardonnay (Navarra)
17075                                                 Trefethen 2002 Riesling (Napa Valley)
17202                                          Arauco 2012 Sauvignon Blanc (Central Valley)
4958                                                    Mariposa 2016 Chardonnay (San Juan)
518                                                  Marco Zunino 2014 Chardonnay (Mendoza)
14569      Marqués de la Concordia 2012 MM Reserva de la Familia Brut Rosé Sparkling (Cava)
10217    Harbes Family Vineyard 2014 Steel Fermented Chardonnay (North Fork of Long Island)
3523                          Binyamina 2009 Reserve Kosher Sauvignon Blanc (Upper Galilee)
2234                                     Ernst Gouws & Co. 2011 Chardonnay (Coastal Region)
4346                                            LaVelle 2010 Pinot Gris (Willamette Valley)
2011                      Cambridge & Sunset 2013 California Series Chardonnay (

In [38]:
rec5 = sample.loc[[17075, 17202, 518, 4958, 10217, 4904, 16708, 18538, 14569, 3523],['description', 'title']]
rec5

,description,title
17075,"Dry, bright and light, with simple, modest fruit flavors and a short clean finsih.",Trefethen 2002 Riesling (Napa Valley)
17202,Punchbowl sweetness and pineapple aromas come in front of a chunky palate with banana and oily green-fruit flavors. A short finish is heavy before it fades.,Arauco 2012 Sauvignon Blanc (Central Valley)
518,"Aromas of banana, peach and citrus fruits are clean. This is round and simple in feel, with flavors of apple and peach that are slightly briny. On the finish, tropical fruit flavors fade to melony and nutty.",Marco Zunino 2014 Chardonnay (Mendoza)
4958,"This clean, mild quaffer offers basic white-fruit aromas and a light-bodied, fresh palate. Apple and mild citrus flavors are short on the finish, which takes on an apricot flavor as it fades away.",Mariposa 2016 Chardonnay (San Juan)
10217,"Notes of apple, yellow peach and pear are savory in tone in this unoaked Chardonnay. Dry in style, it's delicately concentrated with soft tangerine acidity. The finish is brisk but clean.",Harbes Family Vineyard 2014 Steel Fermented Chardonnay (North Fork of Long Island)
4904,"With Chardonnay from Provence the dominant partner in this blend with Burgundian Chardonnay, this wine is soft and unoaked, with melon and yellow fruit flavors. It has light acidity, enough to give the wine freshness. It is ready to drink.",Simonnet-Febvre 2014 100 Series Chardonnay (Vin de France)
16708,"The faux French name and labeling notwithstanding, this is a no-frills Oregon Chardonnay, fruity and clean. Simple flavors of melon and apple carry the palate, with no suggestion of oak.",Domaine Loubejac 2014 Willamette Chardonnay (Willamette Valley)
18538,"Basic tropical-fruit aromas settle on pineapple. This unoaked chardonnay is melony on the palate, with ample acidity. Heavy, ripe flavors of peach, melon and pear turn towards mango on the finish.",Zolo 2016 Sustainably Farmed Estate Grown and Bottled Unoaked Chardonnay (Mendoza)
14569,This round yet direct Cava offers stone-fruit aromas that are upright and clean. Simple citrus and nectarine flavors hold steady on the finish.,Marqués de la Concordia 2012 MM Reserva de la Familia Brut Rosé Sparkling (Cava)
3523,Light weight and somewhat sweet with flavors of gummy peach candy and tropical fruit juice. Short and simple; drink now.,Binyamina 2009 Reserve Kosher Sauvignon Blanc (Upper Galilee)


In [39]:
rec5_text = rec5['description']

cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize, max_features = 2000)

cvec.fit(rec5_text)

new_df  = pd.DataFrame(cvec.transform(rec5_text).todense(),
             columns=cvec.get_feature_names()).sum(axis=0)

new_df.transpose().sort_values(0, ascending=False).transpose().head(15)

fruit     9
clean     6
finish    6
simpl     5
aroma     5
peach     5
light     4
appl      4
palat     4
short     4
citrus    3
fade      3
melon     3
acid      3
unoak     3
dtype: int64

#### Recommendation #3: Red boxed wine

In this case, the input wine was a red and all the recommendations brought back are also reds. This time the recommender is focused on the word "Blend" and is bringing back other blends, but also that are Dry, Fruity, and with Blackberries, and Cherries. The word "Everyday" is also common to four of recommendations. 

In [40]:
get_recommendations("Black Box 2010 Cabernet Sauvignon (California)")

18491                Black Box 2010 Cabernet Sauvignon (California)
5023          Magistrate 2010 Cabernet Sauvignon (Alexander Valley)
8744                           Cirque du Vin 2009 Red (Paso Robles)
13290                    Novy 2009 Zinfandel (Russian River Valley)
12475               Round Hill 2008 Cabernet Sauvignon (California)
19724                          St. Francis 2005 Red (Sonoma County)
18887        Latah Creek 2012 Fries Vineyard Merlot (Wahluke Slope)
7095                                  SKN 2007 Merlot (Napa Valley)
10478                      Grove Ridge 2011 Chardonnay (California)
16454                       Hahn Estates 2008 Chardonnay (Monterey)
4012     Summers 2010 Summers Ranch Reserve Merlot (Knights Valley)
Name: title, dtype: object

In [41]:
rec2 = sample.loc[[18491, 10473, 16327, 19101, 5023, 8744, 19517, 13290, 18147, 17916, 12475 ],['description', 'title']]
rec2

,description,title
18491,"Quite a good Cabernet, especially at this price. They blended some other varieties, including Zinfandel, Malbec and Petite Sirah, which bring extra layers. It's a fine everyday wine—fruity, spicy, full-bodied and dry.",Black Box 2010 Cabernet Sauvignon (California)
10473,"The blend is intriguing, consisting of Merlot, Zinfandel, Syrah and Petite Sirah, in that order. Each variety brings something to the bottle. It's dry, tannic and rich in blackberries, licorice, dark chocolate and spices, but the alcohol is quite high, giving it a porty finish. Drink up.",Phipps 2010 Status Quo Red (Sonoma County)
16327,"Nottingham Cellars' blend of Petite Sirah, Cabernet Sauvignon, Petit Verdot, Cabernet Franc, Merlot and Syrah, all from the 2009, 2010 and 2011 vintages, is an interesting experiment in blending and makes for a fine, earthy, brooding wine. With so much chewy cranberry-dominated flavor, this is a good food wine, especially for early spring fare.",Nottingham Cellars NV AlphaRed Lot III Red (Central Coast)
19101,"Although this Zinfandel, Carignane and Petite Sirah blend is rustic, the price makes it a good value in a dry, full-bodied red wine. With its strong tannins and flavors of currants and black pepper, it will be nice with Mexican food.",Laurel Glen 2008 Reds Red (Lodi)
5023,"Here's a sound, everyday Cabernet. It's dry and full-bodied, with some good blackberry, raisin and oak flavors. A nice buy for the price.",Magistrate 2010 Cabernet Sauvignon (Alexander Valley)
8744,"Made from a whole bunch of varieties—including Zinfandel and Cabernet Sauvignon—this is a soft, fruity wine. It has lush flavors of blackberries, cherries and spices.",Cirque du Vin 2009 Red (Paso Robles)
19517,"A blend of Syrah, Grenache, Petite Sirah and Mourvedre, this is a full-bodied, smoothly fruity wine for drinking now. It has rich blackberry and cherry flavors, with a hit of sweet, smoky oak.",Cass 2010 Rockin' One Red (Paso Robles)
13290,"A good, everyday Zinfandel, spicy and fruity. It has lots of raspberry and cherry jam, chocolate and pepper flavors that finish a little sweet.",Novy 2009 Zinfandel (Russian River Valley)
18147,"Blended from Syrah, Cabernet Sauvignon and Petite Sirah, this wine is soft and fruity. With ripe blackberry and cherry pie, mocha and licorice flavors, it's easy to drink now.",D'Anbino 2011 Orchestration Red (Paso Robles)
17916,"Berry crumble and licorice star in this full-bodied, hearty and smooth wine, blended with 4% Petite Sirah. Spicy oak and milk chocolate unleash further flavor and spice on the finish.",Dunbar 2012 Estate Grown Zinfandel (Sonoma Valley)


In [42]:
rec2_text = rec2['description']

In [43]:
cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize, max_features = 2000)

cvec.fit(rec2_text)

new_df  = pd.DataFrame(cvec.transform(rec2_text).todense(),
             columns=cvec.get_feature_names()).sum(axis=0)

new_df.transpose().sort_values(0, ascending=False).transpose().head(15)

blend         8
good          6
dri           5
cherri        5
fruiti        5
blackberri    5
bodi          5
price         4
everyday      4
spice         3
fine          3
chocol        3
finish        3
licoric       3
spici         3
dtype: int64

#### Recommendation 4:   High-end red

Classing it up with a high-end red. All wines brought back are reds, and also interestingly enough several are from the same region in France, Medoc. Common words are Fruit, Acid, Tannin, Ripe, Fresh, Structure and Currant.

In [58]:
get_recommendations("Château Margaux 2013  Margaux")

7252                                             Château Margaux 2013  Margaux
2780                                  Roger et Didier Raimbault 2010  Sancerre
5466                                Château Tour Saint-Joseph 2008  Haut-Médoc
17483                  Louis Latour 2008 Vignes Franches Premier Cru  (Beaune)
6973                VIgnobles Roux 2014 Château La Fôret  (Bordeaux Supérieur)
1858                                    Château Haut-Bellevue 2012  Haut-Médoc
750                                              Château Chantelys 2010  Médoc
16480                                      Château Bernadotte 2014  Haut-Médoc
18878                              J. & F. Lurton 2008 Barco Negro Red (Douro)
14526    Schröder & Schÿler 2012 Chartron la Fleur Merlot-Cabernet  (Bordeaux)
9481          Caves da Montanha 2013 Monte do Enforcado Tinto Red (Alentejano)
Name: title, dtype: object

In [59]:
rec4 = sample.loc[[2780, 5466, 17483, 6973, 1858, 750, 16480, 18878, 14526, 9481],['description', 'title']]
rec4

,description,title
2780,"Smooth and softly textured, this is an attractive red wine. It has ripe berry fruits, gentle tannins that are enough to bring out the wood and spice flavors. Give the wine two years.",Roger et Didier Raimbault 2010 Sancerre
5466,"With the structured influence of Cabernet Sauvignon, this is a wine that brings out the freshness of 2008, along with its firm tannins. It is still a young wine with acidity and tannins dominating the fruit.",Château Tour Saint-Joseph 2008 Haut-Médoc
17483,"Soft and ripe wine, with gentle tannins and fresh acidity. There is an edge of bitterness that comes between the fruit and the structure.",Louis Latour 2008 Vignes Franches Premier Cru (Beaune)
6973,"This ripe, juicy wine offers blackberry flavors and young acidity. It is developing balance, fresh fruits and supporting tannins. The structure is integrating into the fruit, which will dominate if you wait to drink until 2017. The blend is 70% Merlot and 30% Cabernet Sauvignon.",VIgnobles Roux 2014 Château La Fôret (Bordeaux Supérieur)
1858,"While juicy Merlot dominates this wine, the structure comes from the perfumed Cabernet Sauvignon that brings out the tannins, while the fruit is packed with blackberry flavors. This ripe wine needs to develop over the next few years. Drink from 2017.",Château Haut-Bellevue 2012 Haut-Médoc
750,"This is a fruity, Cabernet Sauvignon-dominated wine, with tannins that support and balance the black currant flavor. Wood aging lightly brushes the structure.",Château Chantelys 2010 Médoc
16480,"The great dominance of Cabernet Sauvignon in the blend has given this wine both a fine structure and crisp swathes of black-currant fruit. It is a firm wine at this stage, although the fine fruitiness will become an important part of the taste. The estate is named after one of Napoleon's generals who became King of Sweden. Drink from 2021.",Château Bernadotte 2014 Haut-Médoc
18878,"A lean style bringing out acidity from the juicy black currant fruits. It has definite tannins supporting the fruit, while it finishes with fresh, perhaps too fresh, acidity.",J. & F. Lurton 2008 Barco Negro Red (Douro)
14526,"This soft, ripe wine is a blend of 70% Merlot and 30% Cabernet Sauvignon. With black-currant fruit and bright acidity, the wine is fruity and ready to drink.",Schröder & Schÿler 2012 Chartron la Fleur Merlot-Cabernet (Bordeaux)
9481,"This is a soft, ripe wine, full of fresh fruit. It is a warm blend of Aragonez and Cabernet Sauvignon, balancing tannins and black currant flavors. It is ready to drink.",Caves da Montanha 2013 Monte do Enforcado Tinto Red (Alentejano)


In [67]:
rec4_text = rec4['description']

cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize, max_features = 2000)

cvec.fit(rec4_text)

new_df  = pd.DataFrame(cvec.transform(rec4_text).todense(),
             columns=cvec.get_feature_names()).sum(axis=0)

new_df.transpose().sort_values(0, ascending=False).transpose().head(15)

fruit       11
tannin       9
acid         6
ripe         6
fresh        6
structur     6
currant      5
domin        5
black        5
bring        4
blend        4
soft         4
support      3
juici        3
balanc       3
dtype: int64

#### Recommendation #5: Champagne

This time I chose a French sparkling wine and the recommender is bringing back several different varietals. The common words are Light, Mouthfeel (which is a real term used to describe the sensation of wine in the mouth), Soft, Refreshing, Rustic, Herbal, and Berry. 

In [76]:
get_recommendations("Kriter NV Eclat Carmin Rosé Sparkling (France)")

878                                   Kriter NV Eclat Carmin Rosé Sparkling (France)
12138                             Ugly Duckling 2010 Cabernet Sauvignon (California)
14647            Armanino Family Cellars 2012 The Ringneck Pinot Noir (Sonoma Coast)
17842                    Proulx 2013 Resistance Willow Creek Farm Rosé (Paso Robles)
4779           Hayman & Hill 2005 Reserve Selection Cabernet Sauvignon (Napa Valley)
547                                               Anne Amie 2012 Pinot Gris (Oregon)
13177            Tres Palacios 2013 Family Vintage Cabernet Sauvignon (Maipo Valley)
15542                                       Croze 2013 Cabernet Sauvignon (Oakville)
9857                              TerraNoble 2006 Gran Reserva Merlot (Maule Valley)
5333     Castillo De Feliciana 2010 Phinny Hill Vineyard Malbec (Horse Heaven Hills)
2138                                Tolosa 2014 Estate Sauvignon Blanc (Edna Valley)
Name: title, dtype: object

In [77]:
rec3 = sample.loc[[878, 12138, 14647, 17842, 13421, 4779, 11471, 5333, 547, 13177, 4938],['description', 'title']]
rec3

,description,title
878,"Light and refreshing, with hints of berries and citrus alongside some herbal elements. The bead is a little on the large side, making the mouthfeel a bit rustic, but it'll do fine at the season's big holiday gatherings.",Kriter NV Eclat Carmin Rosé Sparkling (France)
12138,"A big letdown after the 2009, this is a simple, rustic wine. It's dry and acidic, with thin berry, herb and spice flavors.",Ugly Duckling 2010 Cabernet Sauvignon (California)
14647,"Juicy and big-bodied, this is a balanced expression of earthy black cherry and well-honed structure. Soft and lush on the palate, it'll appeal to fans who like a bit more meat on their Pinot bones, yet appreciate the fine texture of the tannins.",Armanino Family Cellars 2012 The Ringneck Pinot Noir (Sonoma Coast)
17842,"This bottling starts with a pretty nose of peach, melon and nectarine. It's more about the crisp, refreshing mouthfeel than anything, and would partner well with lightly seasoned shellfish or soft cheese.",Proulx 2013 Resistance Willow Creek Farm Rosé (Paso Robles)
13421,"Tastes soft and a little tired at the age of six-plus years, with pie-filling raspberry, cherry and spice flavors. A certain dry complexity makes it interesting, but its lifespan is largely exhausted.",Collier Falls 2004 Hillside Estate Cabernet Sauvignon (Dry Creek Valley)
4779,"A little rough around the edges, with some green tannins and minty, herbal flavors. You get some candied cherries and currants, but the wine is pretty rustic.",Hayman & Hill 2005 Reserve Selection Cabernet Sauvignon (Napa Valley)
11471,"Soft notes of apple are bedded on a creamy, biscuity background of mellow richness. The structure of Pinot Noir shines through and makes for an elegant if not overly fruited wine. Fine bubbles make for a very smooth mouthfeel, all the way to the dry, refreshing finish.",Willm NV Blanc de Noirs Brut Pinot Noir (Alsace)
5333,This lightly aromatic Malbec has notes of plum and wood spice. The tannins are rustic and the flavors light.,Castillo De Feliciana 2010 Phinny Hill Vineyard Malbec (Horse Heaven Hills)
547,"This is a young, yeasty wine with a scent of fresh citrus. Lemon and grapefruit are the main flavors, with a textural, refreshing mouthfeel and a hint of minerality.",Anne Amie 2012 Pinot Gris (Oregon)
13177,"Although this is herbal on the nose, there are solid berry aromas to go with a strong whiff of black olive. A friendly, mediumbodied palate and lightly herbal berry and plum flavors end with minty notes and a good mouthfeel.",Tres Palacios 2013 Family Vintage Cabernet Sauvignon (Maipo Valley)


In [78]:
rec3_text = rec3['description']

cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize, max_features = 2000)

cvec.fit(rec3_text)

new_df  = pd.DataFrame(cvec.transform(rec3_text).todense(),
             columns=cvec.get_feature_names()).sum(axis=0)

new_df.transpose().sort_values(0, ascending=False).transpose().head(15)

light        7
mouthfeel    6
soft         5
rustic       4
herbal       4
refresh      4
berri        4
note         3
cherri       3
big          3
tannin       3
dri          3
fine         3
spice        3
littl        3
dtype: int64

## Section 3. Metric to evaluate the "success" of the recommender.

While the recommender appears to be working as expected in that it is bringing back recommended wines with lots of words in common, it is hard to tell by just eye-balling the individual recommendations. 
So, I compiled the top 10 words used in each of the 10 recommendated wines, and then took the average. As said above, if the recommender is successful, it will recommend wines that have lots of words in common. 

Just looking at the top 10 words in each recommendtion, and over a sample of 100 randomly selected wines, the average is 6.6 words in common. 

In [86]:
sample_titles = df['title'].sample(n=100, random_state=42)
all_recs_matrix = pd.DataFrame()

for title in sample_titles:
    rec_titles = get_recommendations(title)
    rec_df = pd.DataFrame(rec_titles).reset_index()
    rec = sample.loc[rec_df['index'],['description']]
    rec_text = rec['description']
    cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize)
    cvec.fit(rec_text)

    test_df = pd.DataFrame(cvec.transform(rec_text).todense().sum(axis=0)).transpose().sort_values(0, ascending=False).head(10).reset_index()
    rec_matrix = test_df.drop(['index'], axis=1).transpose()
    
    all_recs_matrix = all_recs_matrix.append(rec_matrix, ignore_index=True)

all_recs_matrix['mean'] = all_recs_matrix.mean(axis=1)
all_recs_matrix.head(10)

,0,1,2,3,4,5,6,7,8,9,mean
0,10,10,10,9,7,7,6,6,5,4,7.4
1,11,10,8,7,6,4,4,4,4,3,6.1
2,11,9,9,6,5,5,4,4,4,3,6.0
3,9,9,8,8,7,7,6,6,6,5,7.1
4,11,11,10,9,8,7,6,6,5,4,7.7
5,12,10,10,9,9,8,7,7,6,6,8.4
6,15,11,11,11,10,6,6,6,6,5,8.7
7,12,11,9,6,6,5,5,4,4,4,6.6
8,8,7,7,7,5,4,4,4,4,3,5.3
9,9,7,6,5,5,5,5,5,5,5,5.7


In [81]:
all_recs_matrix['mean'].mean()

6.644000000000002

## Section 4. Recommenders for Red and White wines only

It just bugged me that if a user would input a white wine, they might get recommended a red. So, I created separate dataframes for red wines and white wines and duplicated the code to create recommenders for just red or white wines. 

I also re-did the accuracy score I created above, and the average across samples of 100 wines for both red and white recommendations is 6.7 words in common. 

In [46]:
RW_dict = {'Pinot Noir': 'Red', 
                  'Cabernet Sauvignon': 'Red', 
                  'Red Blend': 'Red',
                  'Bordeaux-style Red Blend':'Red',
                  'Syrah': 'Red', 
                  'Merlot':'Red', 
                  'Zinfandel':'Red', 
                  'Malbec': 'Red', 
                  'Nebbiolo': 'Red', 
                  'Sangiovese': 'Red',
                  'Portuguese Red': 'Red', 
                  'Tempranillo': 'Red', 
                  'Rhône-style Red Blend': 'Red', 
                  'Chardonnay': 'White', 
                  'Riesling':'White', 
                  'Sauvignon Blanc': 'White', 
                  'Rosé': 'White', 
                  'White Blend': 'White', 
                  'Sparkling Blend': 'White', 
                  'Pinot Gris': 'White'}

df['red_or_white']= df['variety'].map(RW_dict)

In [47]:
df['red_or_white'].value_counts()

Red      49452
White    24239
Name: red_or_white, dtype: int64

In [48]:
reds = df[df['red_or_white']=='Red'].copy()
reds.shape

(49452, 15)

In [49]:
whites = df[df['red_or_white']=='White'].copy()
whites.shape

(24239, 15)

### Red Recommender

In [50]:
red_sample = reds.sample(n=20000, random_state=42)

In [51]:
red_sample = red_sample.reset_index()

In [52]:
red_sample.shape

(20000, 16)

In [53]:
#on a sample of 20000
red_tf = TfidfVectorizer(stop_words=stop_words,tokenizer=tokenize, max_features = 2000)
red_tf_matrix = tf.fit_transform(red_sample['description'])

In [54]:
red_cosine_sim = linear_kernel(red_tf_matrix, red_tf_matrix)

In [55]:
red_indices = pd.Series(red_sample.index, index=red_sample['title']).drop_duplicates()

In [56]:
pick_one_red = ''
def get_red_choices(key):
    
    choices = red_sample['title']    
    
    matches = (process.extract(key, choices, scorer = fuzz.ratio, limit = 5))
    print("\n Here are the 5 closest matches...Pick one for the recommender to use. \n")
    print("\n".join(map(str, matches)))
    
    global pick_one_red
    pick_one_red = input("Enter wine here: ")
    
    print("\n Okay, here are 10 other wines that are like", pick_one_red.title())
    
    def fuzzy_match(key):
    
        choices = red_sample['title']    

        #returns the top matching choice
        return process.extractOne(key, choices, scorer = fuzz.token_sort_ratio, score_cutoff = 50)[0]

    #assigning the top choice to title since that is an input for the next function
    red_title = fuzzy_match(pick_one_red)

    def get_red_recommendations(red_title, cosine_sim=red_cosine_sim):

        # Get the index of the wine that matches the title
        idx = red_indices[red_title]

        # Get the pairwsie similarity scores of all wines with that wine
        sim_scores = list(enumerate(red_cosine_sim[idx]))

        # Sort the wines based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the 10 most similar wines
        sim_scores = sim_scores[0:11] #included the 0-th index so I can see the original wine too

        # Get the wines indices
        wine_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar wines
        return sample['title'].iloc[wine_indices]

    results = get_red_recommendations(red_title)
    print(results)

In [57]:
red_wine = input('What wine would you like to search for?')
get_red_choices(red_wine)

What wine would you like to search for?black box

 Here are the 5 closest matches...Pick one for the recommender to use. 

('Black Box 2011 Malbec (Mendoza)', 46, 10628)
('Black Box 2008 Merlot (California)', 43, 8030)
('Black Box 2010 Sweet Red (California)', 40, 9656)
('Coiled 2010 Black Mamba Red', 39, 4514)
('Black Cabra 2015 Malbec (Mendoza)', 39, 19379)
Enter wine here: black box malbec

 Okay, here are 10 other wines that are like Black Box Malbec
10628                                      Craggy Range 2012 Aroha Pinot Noir (Martinborough)
18424                                                Zantho 2016 Sauvignon Blanc (Burgenland)
13508                       J. Rickards 2010 Ancestor Selections Zinfandel (Alexander Valley)
3024                                           Mission Hill 2011 Oculus Red (Okanagan Valley)
19947                                     Davis Family 2009 Pinot Noir (Russian River Valley)
17176                                                       Meli 2012 R

### Whites Recommender

In [58]:
white_sample = whites.sample(n=20000, random_state=42)

In [59]:
white_sample = white_sample.reset_index()

In [60]:
white_sample.shape

(20000, 16)

In [61]:
#on a sample of 20000
white_tf = TfidfVectorizer(stop_words=stop_words,tokenizer=tokenize, max_features = 2000)
white_tf_matrix = tf.fit_transform(white_sample['description'])

In [62]:
white_cosine_sim = linear_kernel(white_tf_matrix, white_tf_matrix)

In [63]:
white_indices = pd.Series(white_sample.index, index=white_sample['title']).drop_duplicates()

In [64]:
pick_one_white = ''
def get_white_choices(key):
    
    choices = white_sample['title']    
    
    #gets the top five closest matches ands asks the user to pick one
    matches = (process.extract(key, choices, scorer = fuzz.ratio, limit = 5))
    print("\n Here are the 5 closest matches...Pick one for the recommender to use. \n")
    print("\n".join(map(str, matches)))
    
    global pick_one_white
    pick_one_white = input("Enter wine here: ")
    
    print("\n Okay, here are 10 other wines that are like", pick_one_white.title())
    
    def fuzzy_match(key):
    
        choices = white_sample['title']    

        #returns the top matching choice
        return process.extractOne(key, choices, scorer = fuzz.token_sort_ratio, score_cutoff = 50)[0]

    #assigning the top choice to title since that is an input for the next function
    white_title = fuzzy_match(pick_one_white)

    def get_white_recommendations(white_title, cosine_sim=white_cosine_sim):

        # Get the index of the wine that matches the title
        idx = white_indices[white_title]

        # Get the pairwsie similarity scores of all wines with that wine
        sim_scores = list(enumerate(white_cosine_sim[idx]))

        # Sort the wines based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the 10 most similar wines
        sim_scores = sim_scores[0:11] #included the 0-th index so I can see the original wine too

        # Get the wines indices
        wine_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar wines
        return white_sample['title'].iloc[wine_indices]

    white_results = get_white_recommendations(white_title)
    print(white_results)

In [65]:
white_wine = input('What wine would you like to search for?')
get_white_choices(white_wine)

What wine would you like to search for?iron horse brut

 Here are the 5 closest matches...Pick one for the recommender to use. 

('Iron Horse 2013 Brut X Sparkling (Green Valley)', 49, 3738)
('Iron Horse 2011 Brut X Sparkling (Green Valley)', 49, 4254)
('Iron Horse 2006 Brut X Sparkling (Green Valley)', 49, 5480)
('Iron Horse 2010 Brut X Sparkling (Green Valley)', 49, 19686)
('Iron Horse 2000 Brut LD Sparkling (Green Valley)', 48, 1412)
Enter wine here: iron horse 2013 brut sparkling

 Okay, here are 10 other wines that are like Iron Horse 2013 Brut Sparkling
3738                                    Iron Horse 2013 Brut X Sparkling (Green Valley)
11472                                                     Domaine Ellevin 2014  Chablis
19957    Lucie 2015 Dutton Ranch Dutton Hill Vineyard Chardonnay (Russian River Valley)
9662                        Domaines Ott 2007 Château de Selle Rosé (Côtes de Provence)
4475                                               Wunsch & Mann 2011 Riesling (Al

### Metric for red white recommendations

In [66]:
def get_red_recommendations(title, cosine_sim=red_cosine_sim):
    # Get the index of the wine that matches the title
    idx = red_indices[title]

    # Get the pairwsie similarity scores of all wines with that wine
    sim_scores = list(enumerate(red_cosine_sim[idx]))

    # Sort the wines based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar wines
    sim_scores = sim_scores[0:11] #included the 0-th index so I can see the original wine too

    # Get the wines indices
    wine_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar wines
    return red_sample['title'].iloc[wine_indices]


In [67]:
sample_red_titles = reds['title'].sample(n=100, random_state=42)
all_red_recs_matrix = pd.DataFrame()

for title in sample_red_titles:
    rec_titles = get_red_recommendations(title)
    rec_df = pd.DataFrame(rec_titles).reset_index()
    rec = red_sample.loc[rec_df['index'],['description']]
    rec_text = rec['description']
    cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize)
    cvec.fit(rec_text)

    test_df = pd.DataFrame(cvec.transform(rec_text).todense().sum(axis=0)).transpose().sort_values(0, ascending=False).head(10).reset_index()
    rec_matrix = test_df.drop(['index'], axis=1).transpose()
    
    all_red_recs_matrix = all_red_recs_matrix.append(rec_matrix, ignore_index=True)

all_red_recs_matrix['mean'] = all_red_recs_matrix.mean(axis=1)
all_red_recs_matrix.head(10)

,0,1,2,3,4,5,6,7,8,9,mean
0,7,6,6,5,5,5,5,5,4,4,5.2
1,10,10,10,9,7,7,7,6,5,5,7.6
2,10,10,9,6,4,4,4,3,3,3,5.6
3,8,7,6,5,4,4,4,4,4,4,5.0
4,5,5,5,5,5,5,5,4,4,4,4.7
5,13,11,11,10,8,7,7,6,6,5,8.4
6,8,7,7,6,5,5,5,4,4,4,5.5
7,9,6,6,6,6,6,5,5,5,4,5.8
8,11,11,7,7,6,5,5,4,4,4,6.4
9,11,9,8,8,7,7,7,7,6,6,7.6


In [68]:
all_red_recs_matrix['mean'].mean()

6.699999999999999

### Metric for white recommendations

In [69]:
def get_white_recommendations(title, cosine_sim=white_cosine_sim):
    # Get the index of the wine that matches the title
    idx = white_indices[title]

    # Get the pairwsie similarity scores of all wines with that wine
    sim_scores = list(enumerate(white_cosine_sim[idx]))

    # Sort the wines based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar wines
    sim_scores = sim_scores[0:11] #included the 0-th index so I can see the original wine too

    # Get the wines indices
    wine_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar wines
    return white_sample['title'].iloc[wine_indices]


In [70]:
get_white_recommendations('Iron Horse 2013 Brut X Sparkling (Green Valley)')

3738                                    Iron Horse 2013 Brut X Sparkling (Green Valley)
11472                                                     Domaine Ellevin 2014  Chablis
19957    Lucie 2015 Dutton Ranch Dutton Hill Vineyard Chardonnay (Russian River Valley)
9662                        Domaines Ott 2007 Château de Selle Rosé (Côtes de Provence)
4475                                               Wunsch & Mann 2011 Riesling (Alsace)
11493                                      Layer Cake 2015 Sauvignon Blanc (California)
6807                                               Casanel 2014 Chardonnay (Middleburg)
16286                                       B Cellars 2014 Blend 23 White (Napa Valley)
12110               Pride Mountain 2015 Vintner Select Cuvée Chardonnay (Sonoma County)
7577                               Dragonette 2012 Rosé (Happy Canyon of Santa Barbara)
19416                                                 Leon Beyer 2001 Riesling (Alsace)
Name: title, dtype: object

In [74]:
rec6 = white_sample.loc[[3738, 11472, 19957, 9662, 4475, 11493, 6807, 16286, 12110, 7577, 19416],['description', 'title']]
rec6

,description,title
3738,"Extra dry in style, this sparkling blend of 68% Pinot Noir and 32% Chardonnay succeeds dramatically in its ability to be salty as a cracker and crisp as a bite of green apple, followed by a lemon chaser. Tart and lengthy, it has persistent acidity and purity that's hard to resist.",Iron Horse 2013 Brut X Sparkling (Green Valley)
11472,"This crisp, green-fruit-flavored wine has refreshing acidity along with a tangy, citrus texture that is immediately attractive. A mineral edge gives this wine extra bite. Drink now.",Domaine Ellevin 2014 Chablis
19957,"Exotically floral on the nose, the prettiness of this wine follows through on the palate, salty and high in acidity, with a tang of lemon peel and lemon verbena. Focused and fresh, it has body and weight within that context, ultimately finding a balance that persists on the lengthy finish.",Lucie 2015 Dutton Ranch Dutton Hill Vineyard Chardonnay (Russian River Valley)
9662,"It's expensive, but there's no denying the high quality of this wine. Almost smoky, mineral-like complexity emerges on the nose, followed by hints of cherry fruit underscored by dramatic freshness on the lengthy finish.",Domaines Ott 2007 Château de Selle Rosé (Côtes de Provence)
4475,"A bitter wine with tart apple and pear flavors, this has a hard edge. Drink now.",Wunsch & Mann 2011 Riesling (Alsace)
11493,"An herbal, earthy aroma is followed by crisp fruit flavors in this medium-bodied wine. It's tart and tangy on the palate, those leafy, green notes persisting through the finish.",Layer Cake 2015 Sauvignon Blanc (California)
6807,"Ripe peach and lemon meet graham cracker on the nose and palate. Medium in weight, with a slightly flabby feel, green apple acidity works hard to maintain balance.",Casanel 2014 Chardonnay (Middleburg)
16286,"The producer blends 57% Chardonnay from the Maldonado Vineyard with 39% Sauvignon Blanc and 4% Viognier for this fresh, floral white. On the palate, apple and lemon flavors are buttressed by a core of laser-like acidity. A fresh pear note lasts long into the finish, proving the wine's ability to be both creamy and crisp.",B Cellars 2014 Blend 23 White (Napa Valley)
12110,"Oaky toast and salty, firm acidity give this wine a flavor profile of lemon, apple and dried herb that's lengthy and full of complexity. Medium bodied, it finds balance between fruit flavors and a savory edge, remaining bright and rich on the palate.",Pride Mountain 2015 Vintner Select Cuvée Chardonnay (Sonoma County)
7577,"Rosé should be dry, delicate and crisp in acidity. This Grenache, Mourvèdre and Syrah blend succeeds on two of those levels, but not dryness. The raspberry fruit has a simple jamminess that's almost dessert-style.",Dragonette 2012 Rosé (Happy Canyon of Santa Barbara)


In [75]:
rec6_text = rec6['description']

cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize, max_features = 2000)

cvec.fit(rec6_text)

new_df  = pd.DataFrame(cvec.transform(rec6_text).todense(),
             columns=cvec.get_feature_names()).sum(axis=0)

new_df.transpose().sort_values(0, ascending=False).transpose().head(15)

acid       7
lemon      6
crisp      6
appl       5
fruit      5
palat      5
lengthi    4
finish     4
fresh      4
green      4
follow     4
medium     3
persist    3
salti      3
bodi       3
dtype: int64

In [76]:
sample_white_titles = whites['title'].sample(n=100, random_state=42)
all_whites_recs_matrix = pd.DataFrame()

for title in sample_white_titles:
    rec_titles = get_white_recommendations(title)
    rec_df = pd.DataFrame(rec_titles).reset_index()
    rec = white_sample.loc[rec_df['index'],['description']]
    rec_text = rec['description']
    cvec = CountVectorizer(stop_words=stop_words, tokenizer=tokenize)
    cvec.fit(rec_text)

    test_df = pd.DataFrame(cvec.transform(rec_text).todense().sum(axis=0)).transpose().sort_values(0, ascending=False).head(10).reset_index()
    rec_matrix = test_df.drop(['index'], axis=1).transpose()
    
    all_whites_recs_matrix = all_whites_recs_matrix.append(rec_matrix, ignore_index=True)

all_whites_recs_matrix['mean'] = all_whites_recs_matrix.mean(axis=1)
all_whites_recs_matrix.head(10)

,0,1,2,3,4,5,6,7,8,9,mean
0,9,9,9,8,8,8,7,7,6,6,7.7
1,9,7,7,6,6,5,4,4,4,3,5.5
2,11,9,8,8,7,7,7,6,6,6,7.5
3,12,10,9,9,8,8,7,7,6,6,8.2
4,10,10,9,9,8,7,7,7,7,6,8.0
5,11,11,9,8,6,6,5,4,4,4,6.8
6,10,10,9,8,7,7,7,6,6,6,7.6
7,7,6,6,6,6,4,4,4,4,4,5.1
8,14,6,5,5,5,5,4,3,3,3,5.3
9,13,9,9,9,7,6,5,5,5,5,7.3


In [77]:
all_whites_recs_matrix['mean'].mean()

6.738999999999999